In [289]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import html
import re
import json
import time

In [2]:
#структура сайта
main_url = "http://marspb.ru"
types = ['tram', 'trolleybus', 'bus', 'marshrutka']

In [214]:
#scrapper
def getRoutsHrefFromPage(page_href):
    url = main_url + page_href
    
    r = requests.get(url)
    text = r.text
    soup = BeautifulSoup(text, "lxml")
    
    rout_items = soup.find_all('div', {'class': 'route-item'})
    hrefs = list(map(lambda rout: rout.find('a').get('href'), rout_items))
    numbers = list(map(lambda rout: rout.find('a').text, rout_items))
    
    routs = []
    for i in range(0, len(rout_items)):
        rout = {'href':hrefs[i], 'number': numbers[i]}
        routs.append(rout)
        
    return routs

def getRoutsHrefs():
    routs = []
    
    for mar_type in types:
        href = "/" + mar_type + "/"
         
        routs_part = getRoutsHrefFromPage(href)
        wtf = list(map(lambda rout: rout.update({'type':mar_type}), routs_part))
        
        routs.extend(routs_part)
        
        #Необходимо обходить страницы
        url = main_url + href
        r = requests.get(url)
        text = r.text
        soup = BeautifulSoup(text, "lxml")
        
        page_wrapper = soup.find('div', {'class': 'pager-wrapper'})
        if page_wrapper:
            pages_href = page_wrapper.find_all('a')
            pages = list(map(lambda x : x.get('href'), pages_href))
            
            for page in pages:
                routs_part = getRoutsHrefFromPage(page)
                wtf = list(map(lambda rout: rout.update({'type':mar_type}), routs_part))
                routs.extend(routs_part)
    
    return(routs)

def getRout(rout):
    url = main_url + rout['href']
    r = requests.get(url)
    text = r.text
    soup = BeautifulSoup(text, 'lxml')
    
    rout_text = soup.find_all('script')[-1].text
    rout_text = re.sub("^\s+|\n|\r|\t|\s+$", '', rout_text)
    rout_text = re.sub('.*Route: ', '', rout_text)
    rout_text = re.sub(',MapIconsDirectory.*', '', rout_text)
    
    rout_dict = json.loads(rout_text)
    rout_pf = rout_dict['pf'] if 'pf' in rout_dict else {}
    rout_pb = rout_dict['pb'] if 'pb' in rout_dict else {}
    
    rout.update({'pf':rout_pf, 'pb': rout_pb})
    
    return rout
    

def getAllRouts():    
    routs = getRoutsHrefs()
    for rout in routs:
        getRout(rout)
        #time.sleep(1)
        
        
    return routs

In [279]:
routs = getAllRouts()

In [282]:
rout_pf = list(map(lambda r: {"href":r["href"], "number": r["number"], "direction": "f", "type": r['type'], "rout": r['pf']}, routs))
rout_pb = list(map(lambda r: {"href":r["href"], "number": r["number"], "direction": "b", "type": r['type'], "rout": r['pb']}, routs))

In [283]:
routs = []
routs.extend(rout_pd)
routs.extend(rout_pb)

In [284]:
#convert to geojson
geojson = {
    "type": "FeatureCollection",
    
    "features": [
    {
        "type": "Feature",
        "properties": {
            "href" : r["href"],
            "number" : r["number"],
            "direction": r["direction"],
            "type": r["type"]
        },
        "geometry" : {
            "type": "LineString",
            "coordinates": [[rc["lng"], rc["lat"]] for rc in r['rout']],
            },
        
     } for r in routs]
}

In [291]:
output = open("public_transport_routs_2.geojson", 'w')
json.dump(geojson, output)

In [288]:
geojson['features'][-1]

{'geometry': {'coordinates': [[30.379977, 59.811255],
   [30.382656, 59.809641],
   [30.384505, 59.808628],
   [30.384932, 59.808252],
   [30.386586, 59.805248],
   [30.387121, 59.804315],
   [30.388034, 59.8045],
   [30.387612, 59.805192],
   [30.387256, 59.805886],
   [30.387061, 59.806395],
   [30.387039, 59.807304],
   [30.385407, 59.810575],
   [30.385028, 59.810829],
   [30.384357, 59.810886],
   [30.383495, 59.810693],
   [30.382903, 59.810715],
   [30.38176, 59.81165],
   [30.380973, 59.812123],
   [30.380209, 59.812456],
   [30.376094, 59.814012],
   [30.374576, 59.814681],
   [30.370118, 59.817112],
   [30.366079, 59.819224],
   [30.358785, 59.823151],
   [30.348419, 59.828728],
   [30.349157, 59.829092],
   [30.351532, 59.830203],
   [30.351148, 59.831474],
   [30.3509, 59.831933],
   [30.349423, 59.832753]],
  'type': 'LineString'},
 'properties': {'direction': 'b',
  'href': '/marshrutka/k479',
  'number': 'К479',
  'type': 'marshrutka'},
 'type': 'Feature'}